In [ ]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200)
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import sys
sys.path.append('C:/Users/Petr.Yurchenko/McKinsey_Severstal_Hack_2022')

import warnings
warnings.filterwarnings("ignore")

from server2022.src.train_val import *
from server2022.src.model_features import *
from server2022.data.processed.prepare_data import *

## All features

In [ ]:
df = create_df_0years_known()

In [ ]:
feat_gen = Feature_gen(-4, -1)
df = feat_gen.preprocessing_before_fitting(df)
df = df.loc[:, [col for col in df.columns if 'Факт' not in col]]

In [ ]:
df.head(3)

In [ ]:
model = CatBoostClassifier(iterations=500,
                           depth=5, 
                           l2_leaf_reg=5, 
                           learning_rate=0.01433,
                           loss_function='Logloss',
                           use_best_model=False,
                           eval_metric='AUC',
                           verbose=False,
                           class_weights=[1, 0.0006],
                           random_seed=RANDOM_STATE,
                          )

make_report(model, df, treshold=0.5, to_file=False);

X_train, X_test, y_train, y_test = data_split(df)
probas = model.predict_proba(X_test)[:, 1]

best = 0
xx = 0
for x in sorted(probas):
    pr = precision_score(y_test, probas >= x)
    rl = recall_score(y_test, probas >= x)
    cur = f1_score(y_test, probas >= x)
    if cur >= best:
        best = cur
        xx = x

make_report(model, df, treshold=xx, to_file=True, file_path='../reports/report_0_years.csv', comment=f'all_features')

## No features